# Tree Methods Consulting Project 

You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beelive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one!
Use Machine Learning with RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled.
___

**Think carefully about what this problem is really asking you to solve. While we will use Machine Learning to solve this, it won't be with your typical train/test split workflow. If this confuses you, skip ahead to the solution code along walk-through!**
____

# Good Luck!

Many machine learning models produce some sort of coefficient value for each feature involved, indicating their "importance" or predictive power. Remember back to the documentation lecture for this section of the course!

We mentioned the these tree method classifiers had a .featureImportances attribute available!

So we can create a model, fit it on all the data, and then check which feature (preservative) was causing the spoilage!

There are many different ways to solve this problem, including just using "pure" statistics instead of a machine learning model.

Hopefully this consulting project shows how we can apply machine learning in a different way from previous examples. In this case we don't really care about test/train splits or deployments. We care about the relationship between the feature and the label.

What we really want to understand is the fundamental relationship between each feature column and the label itself.

In [19]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('tree_consult').getOrCreate()

In [20]:
data = spark.read.csv('dog_food.csv', inferSchema=True, header=True)

In [21]:
data.head(1)

[Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0)]

We are going to perform a random forest classifier model on the data itself.

In [22]:
from pyspark.ml.feature import VectorAssembler

In [23]:
data.columns

['A', 'B', 'C', 'D', 'Spoiled']

In [24]:
assembler = VectorAssembler(inputCols=['A','B','C','C'], outputCol='features')

In [25]:
output = assembler.transform(data)

In [26]:
from pyspark.ml.classification import RandomForestClassifier

In [27]:
rfc = RandomForestClassifier(labelCol='Spoiled', featuresCol='features')

In [28]:
output.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)
 |-- features: vector (nullable = true)



In [29]:
final_data = output.select('features','Spoiled')

In [30]:
final_data.show(5)

+-------------------+-------+
|           features|Spoiled|
+-------------------+-------+
|[4.0,2.0,12.0,12.0]|    1.0|
|[5.0,6.0,12.0,12.0]|    1.0|
|[6.0,2.0,13.0,13.0]|    1.0|
|[4.0,2.0,12.0,12.0]|    1.0|
|[4.0,2.0,12.0,12.0]|    1.0|
+-------------------+-------+
only showing top 5 rows



In [31]:
# let's train our classifier
# create model
rfc_model = rfc.fit(final_data)

In [32]:
# request the features importance of this model
final_data.head(1)

[Row(features=DenseVector([4.0, 2.0, 12.0, 12.0]), Spoiled=1.0)]

In [33]:
rfc_model.featureImportances

SparseVector(4, {0: 0.0107, 1: 0.0051, 2: 0.3462, 3: 0.638})

In [ ]:
Feature at index 3 (Chemical D) is the most important feature for the early spoilage.